In [1]:
#importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


In [10]:
#open the browser
driver=webdriver.Chrome()

#load the webpage
driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")
time.sleep(3)
driver.maximize_window()
wait_time = WebDriverWait(driver, 20)

def get_andhra_bus_data(xpath):
    bus_links = []  # Holds all the bus URLs
    bus_routes = []  # Holds all the route names
    
    for page_num in range(1, 6):
        route_elements = driver.find_elements(By.XPATH, xpath)
        bus_links.extend([element.get_attribute("href") for element in route_elements])
        bus_routes.extend([element.text for element in route_elements])
        
        try:
            pagination_table = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]'))
            )
            next_page_button = pagination_table.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()="{page_num + 1}"]')
            
            # Scroll the next page button into view
            ActionChains(driver).move_to_element(next_page_button).perform()
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable(next_page_button))
            next_page_button.click()
            
        except NoSuchElementException:
            print(f"End of pages detected at page {page_num}")
            break
        except ElementClickInterceptedException:
            print(f"Could not click the next page button at page {page_num}, trying again.")
            time.sleep(1)  # Wait a moment before retrying
            next_page_button.click()
    
    return bus_links, bus_routes




In [11]:
# change dataframe to csv
path=r"C:\Users\Vanathi\Documents\redbus scraped details\andhra_df.csv"
andhra_df.to_csv(path,index=False)

In [12]:
# Run the function to get the bus links and routes
bus_links, bus_routes = get_andhra_bus_data("//a[@class='route']")

# Now create the DataFrame with the returned variables
andhra_df = pd.DataFrame({
    "Bus_Route_Name": bus_routes,  # variable name from ROUTE_ANDHRA
    "Bus_Route_Link": bus_links   # variable name from LINKS_ANDHARA
})

# Display the DataFrame
print(andhra_df)


End of pages detected at page 5
                             Bus_Route_Name  \
0                   Hyderabad to Vijayawada   
1                   Vijayawada to Hyderabad   
2                     Bangalore to Tirupati   
3                       Bangalore to Kadapa   
4                 Kakinada to Visakhapatnam   
5                       Hyderabad to Ongole   
6                       Ongole to Hyderabad   
7   Bangalore to Anantapur (andhra pradesh)   
8    Bangalore to Chittoor (Andhra Pradesh)   
9    Chittoor (Andhra Pradesh) to Bangalore   
10     Hyderabad to Guntur (Andhra Pradesh)   
11              Vijayawada to Visakhapatnam   
12                Hyderabad to Narasaraopet   
13                      Kadapa to Bangalore   
14                Narasaraopet to Hyderabad   
15                Visakhapatnam to Kakinada   
16                      Bangalore to Kadiri   
17                 Bangalore to Madanapalli   
18                 Madanapalli to Bangalore   
19  Anantapur (andhra prades

In [13]:
df=pd.read_csv(r"C:\Users\Vanathi\Documents\redbus scraped details\andhra_df.csv")

In [14]:
driver_andhra = webdriver.Chrome()
Busname_1 = []
Bustype_1 = []
Starttime_1 = []
Endtime_1 = []
Ratings_1 = []
Total_duration_1 = []
Prices_1 = []
Seats_available_1 = []
Route_names = []
Route_links = []

for i, r in df.iterrows():
    link = r["Bus_Route_Link"]
    routes = r["Bus_Route_Name"]

    # Open the link
    driver_andhra.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    elements = driver_andhra.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # Click elements to view bus details
    try:
        clicks = driver_andhra.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
    time.sleep(2)

    # Scroll through the page to load all bus details
    scrolling = True
    while scrolling:
        # Get the current scroll height
        last_height = driver_andhra.execute_script("return document.body.scrollHeight")

        # Scroll down to the bottom
        driver_andhra.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for new content to load
        time.sleep(5)

        # Get the new scroll height
        new_height = driver_andhra.execute_script("return document.body.scrollHeight")

        # Check if the page height has changed
        if new_height == last_height:
            scrolling = False

    # Extract bus details
    try:
        bus_name = WebDriverWait(driver_andhra, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='travels lh-24 f-bold d-color']"))
        )
        bus_type = driver_andhra.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = driver_andhra.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver_andhra.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver_andhra.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        try:
            rating = driver_andhra.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        except:
            continue
        price = driver_andhra.find_elements(By.XPATH, '//*[@class="fare d-block"]')
        seats = driver_andhra.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for bus in bus_name:
            Busname_1.append(bus.text)
            Route_links.append(link)
            Route_names.append(routes)
        for bus_type_elem in bus_type:
            Bustype_1.append(bus_type_elem.text)
        for start_time_elem in start_time:
            Starttime_1.append(start_time_elem.text)
        for end_time_elem in end_time:
            Endtime_1.append(end_time_elem.text)
        for total_duration_elem in total_duration:
            Total_duration_1.append(total_duration_elem.text)
        for ratings in rating:
            Ratings_1.append(ratings.text)
        for price_elem in price:
            Prices_1.append(price_elem.text)
        for seats_elem in seats:
            Seats_available_1.append(seats_elem.text)
    except Exception as e:
        print(f"Error occurred while extracting details: {e}")
        continue

print("Successfully Completed")



Successfully Completed


In [15]:
# Before converting to DataFrame, check if all lists have the same length
lengths = {
    'Busname_1': len(Busname_1),
    'Bustype_1': len(Bustype_1),
    'Starttime_1': len(Starttime_1),
    'Endtime_1': len(Endtime_1),
    'Total_duration_1': len(Total_duration_1),
    'Prices_1': len(Prices_1),
    'Seats_available_1': len(Seats_available_1),
    'Route_names': len(Route_names),
    'Route_links': len(Route_links),
    'Ratings_1': len(Ratings_1)
}

print(lengths)

# Ensure that all lists have the same length
max_length = max(lengths.values())  # Find the longest list
for lst in [Busname_1, Bustype_1, Starttime_1, Endtime_1, Total_duration_1, Prices_1, Seats_available_1, Route_names, Route_links, Ratings_1]:
    while len(lst) < max_length:
        lst.append(None)  # Or you can append a placeholder like an empty string


{'Busname_1': 863, 'Bustype_1': 863, 'Starttime_1': 863, 'Endtime_1': 863, 'Total_duration_1': 863, 'Prices_1': 863, 'Seats_available_1': 863, 'Route_names': 863, 'Route_links': 863, 'Ratings_1': 220}


In [18]:
# Convert list to DataFrame
data = {
    'Bus_name': Busname_1,
    'Bus_type': Bustype_1,
    'Start_time': Starttime_1,
    'End_time': Endtime_1,
    'Total_duration': Total_duration_1,
    'Price': Prices_1,
    "Seats_Available": Seats_available_1,
    "Ratings": Ratings_1,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_2 = pd.DataFrame(data)

# Convert DataFrame to CSV
path = r"C:\Users\Vanathi\Documents\redbus scraped details\df_buses_2.csv"
df_buses_2.to_csv(path, index=False)

print("Successfully Completed and saved to CSV.")

Successfully Completed and saved to CSV.


In [19]:
b=pd.read_csv(r"C:\Users\Vanathi\Documents\redbus scraped details\df_buses_2.csv")
b

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,APSRTC - 3984,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",00:30,04:35,28h 05m,INR 344,6 Seats available,3.4\n39,https://www.redbus.in/bus-tickets/rajahmundry-...,Rajahmundry to Visakhapatnam
1,APSRTC - 3091,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",00:30,05:45,29h 15m,INR 342,26 Seats available,3.5\n30,https://www.redbus.in/bus-tickets/rajahmundry-...,Rajahmundry to Visakhapatnam
2,APSRTC - 3637,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",00:45,04:30,27h 45m,INR 344,29 Seats available,3.9\n93,https://www.redbus.in/bus-tickets/rajahmundry-...,Rajahmundry to Visakhapatnam
3,APSRTC - 4280,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",00:45,04:40,27h 55m,INR 315,29 Seats available,3.8\n67,https://www.redbus.in/bus-tickets/rajahmundry-...,Rajahmundry to Visakhapatnam
4,APSRTC - 3257,"ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)",01:45,05:25,27h 40m,INR 328,35 Seats available,4.6\n47,https://www.redbus.in/bus-tickets/rajahmundry-...,Rajahmundry to Visakhapatnam
...,...,...,...,...,...,...,...,...,...,...
858,Shree Savariya Travels & Transport,A/C Seater / Sleeper (2+1),09:20,13:05,03h 45m,INR 1199,27 Seats available,NaN,https://www.redbus.in/bus-tickets/ananthapur-t...,Anantapur (andhra pradesh) to Bangalore
859,KKaveri Travels,Non A/C Seater / Sleeper (2+1),23:59,05:00,05h 01m,INR 5000,37 Seats available,NaN,https://www.redbus.in/bus-tickets/ananthapur-t...,Anantapur (andhra pradesh) to Bangalore
860,Morning Star Travels,Volvo Multi Axle B9R A/C Sleeper (2+1),03:45,06:30,02h 45m,INR 990,19 Seats available,NaN,https://www.redbus.in/bus-tickets/ananthapur-t...,Anantapur (andhra pradesh) to Bangalore
861,Morning Star Travels,A/C Sleeper (2+1),04:15,07:00,02h 45m,INR 990,10 Seats available,NaN,https://www.redbus.in/bus-tickets/ananthapur-t...,Anantapur (andhra pradesh) to Bangalore
